# 01_EDA_and_Train.ipynb

**Uplift-моделирование: EDA, препроцессинг, обучение и сохранение**



Uplift-моделирование: построить модель, которая оценивает прирост конверсии (uplift) от отправки предложения конкретному клиенту. Цель — определить клиентов, на которых коммуникация оказывает максимальный положительный эффект, и тем самым оптимизировать бюджет маркетинга.


<u> **Бизнес-ценность проекта:** </u> <p>
***1.*** Сокращение маркетингового бюджета за счёт отправки предложений только «убеждаемым» клиентам<p>
***2.*** Повышение ROI кампаний<p>
***3.*** Улучшение клиентского опыта благодаря персонализированной коммуникации<p>

<u> **Описание датасета:** </u>
https://www.kaggle.com/datasets/davinwijaya/customer-retention <P>
Набор данных содержит информацию о клиентах розничной компании и их реакции на маркетинговые коммуникации. Каждый клиент получил одно из трёх предложений (или не получил ничего), и зафиксирован результат — совершил ли клиент покупку после предложения.

***recency*** — количество месяцев с момента последней покупки клиента <p>
***history*** — общая сумма покупок клиента за всё время (в долларах)<p>
***used_discount*** — использовал ли клиент скидки в прошлом <p>
***used_bogo*** — использовал ли клиент акции типа «Купи один — получи второй бесплатно» в прошлом<p>
***zip_code*** — почтовый индекс клиента (категориальный признак: Suburban, Urban, Rural)<p>
***is_referral*** — пришёл ли клиент по реферальной программе <p>
***channel*** — канал привлечения клиента (Phone, Web, Multichannel)<p>
***offer*** — тип полученного предложения (Discount, Buy One Get One, No Offer)<p>
***conversion*** — целевая переменная (1 — клиент совершил покупку, 0 — нет)<p>

# Import

In [1]:
import yaml
import json
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, KBinsDiscretizer
from sklearn.compose import ColumnTransformer
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.metrics import roc_auc_score
import optuna

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline


In [2]:
# находим корень проекта (там, где лежит config/)
PROJECT_ROOT = Path(__file__).parent.parent if "__file__" in locals() else Path.cwd().parents[0]
# PROJECT_ROOT = Path.cwd().parents[0]

config_path = PROJECT_ROOT / 'config' / 'params2.yml'

# Загружаем конфиг
with config_path.open('r', encoding='utf-8') as f:
    config = yaml.safe_load(f)

# Формируем полный путь к данным
raw_path = PROJECT_ROOT / config['data']['raw_path']

# Загружаем датасет 
df = pd.read_csv(raw_path)

print(f"Датасет загружен: {df.shape[0]} строк, {df.shape[1]} колонок")
print(f"Средняя конверсия: {df['conversion'].mean():.2%}")
df.head()


Датасет загружен: 64000 строк, 9 колонок
Средняя конверсия: 14.68%


,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,offer,conversion
0,10,142.44,1,0,Surburban,0,Phone,Buy One Get One,0
1,6,329.08,1,1,Rural,1,Web,No Offer,0
2,7,180.65,0,1,Surburban,1,Web,Buy One Get One,0
3,9,675.83,1,0,Rural,1,Web,Discount,0
4,2,45.34,1,0,Urban,0,Web,Buy One Get One,0


# EDA 

In [3]:
# Папка для отчетов
report_path = PROJECT_ROOT / config['folders']['report']
report_path.mkdir(parents=True, exist_ok=True)

# Гистограммы ключевых признаков 
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

df['recency'].hist(bins=30, ax=axes[0,0], color='skyblue')
axes[0,0].set_title('Recency')

df['history'].hist(bins=50, ax=axes[0,1], color='lightgreen')
axes[0,1].set_title('History')

df['used_discount'].value_counts().plot(kind='bar', ax=axes[1,0], color='salmon')
axes[1,0].set_title('Used Discount')

df['used_bogo'].value_counts().plot(kind='bar', ax=axes[1,1], color='gold')
axes[1,1].set_title('Used BOGO')

plt.tight_layout()

# Сохраняем гистограммы в файл
hist_path = report_path / 'feature_histograms.png'
fig.savefig(hist_path)
plt.close(fig)  
print(f"Гистограммы сохранены: {hist_path}")

# Корреляционная матрица
corr_cols = ['recency', 'history', 'used_discount', 'used_bogo', 'is_referral', 'conversion']
fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(df[corr_cols].corr(), annot=True, cmap='coolwarm', fmt='.2f', ax=ax)
plt.title('Корреляционная матрица')

# Сохраняем корреляции
corr_path = report_path / 'correlation_matrix.png'
fig.savefig(corr_path)
plt.close(fig)
print(f"Корреляционная матрица сохранена: {corr_path}")


Гистограммы сохранены: /Users/bariatmamaeva/Desktop/uplift-marketing-mlops/report/feature_histograms.png
Корреляционная матрица сохранена: /Users/bariatmamaeva/Desktop/uplift-marketing-mlops/report/correlation_matrix.png


# Features engineering 

In [4]:
# Папка для отчетов
report_path = PROJECT_ROOT / config['folders']['report']
report_path.mkdir(parents=True, exist_ok=True)

# Создание взаимодействий (numeric_interactions)
interaction_cols = []
for interaction in config.get('features', {}).get('numeric_interactions', []):
    if interaction == 'history_discount':
        df['history_discount'] = df['history'] * df['used_discount']
        interaction_cols.append('history_discount')
    elif interaction == 'history_bogo':
        df['history_bogo'] = df['history'] * df['used_bogo']
        interaction_cols.append('history_bogo')
    elif interaction == 'recency_history':
        df['recency_history'] = df['recency'] * df['history']
        interaction_cols.append('recency_history')
    else:
        # Универсальный способ, если в конфиге что-то другое
        parts = interaction.split('_and_') if '_and_' in interaction else interaction.split('_')
        if len(parts) >= 2:
            col_name = '_'.join(parts)
            if all(p in df.columns for p in parts):
                df[col_name] = df[parts[0]] * df[parts[1]]
                interaction_cols.append(col_name)

# Биннинг recency 
recency_bins = config.get('features', {}).get('recency_bins', 5)
recency_strategy = config.get('features', {}).get('recency_bin_strategy', 'uniform')

kbins = KBinsDiscretizer(
    n_bins=recency_bins,
    encode='onehot-dense',
    strategy=recency_strategy
)
binned = kbins.fit_transform(df[['recency']])
bin_cols = [f'recency_bin_{i}' for i in range(recency_bins)]
df[bin_cols] = binned
interaction_cols.extend(bin_cols)

# Treatment
df['treatment'] = (df['offer'] != 'No Offer').astype(int)
interaction_cols.append('treatment')

# Логирование новых признаков
features_log_path = report_path / 'created_features.json'
with open(features_log_path, 'w') as f:
    json.dump(interaction_cols, f, indent=2)

print(f"Новые признаки созданы по конфигу: {interaction_cols}")
print(f"Список новых признаков сохранён: {features_log_path}")


Новые признаки созданы по конфигу: ['history_discount', 'history_bogo', 'recency_history', 'recency_bin_0', 'recency_bin_1', 'recency_bin_2', 'treatment']
Список новых признаков сохранён: /Users/bariatmamaeva/Desktop/uplift-marketing-mlops/report/created_features.json


# Preprocessing

In [5]:
y = df['conversion']
X = df.drop(columns=['offer', 'conversion', 'treatment'])

# Определение колонок по конфигу 
categorical_cols = config.get('features', {}).get('categorical', [])
numeric_cols = [col for col in X.columns if col not in categorical_cols]

# Создание препроцессора 
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), numeric_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_cols)
])

# Применение препроцессинга
X_preprocessed = preprocessor.fit_transform(X)
print(f"Признаков после препроцессинга: {X_preprocessed.shape[1]}")

# Сохранение (MLOps)
models_path = PROJECT_ROOT / config['folders']['models']
models_path.mkdir(parents=True, exist_ok=True)

preprocessor_path = models_path / 'preprocessor.joblib'
joblib.dump(preprocessor, preprocessor_path)
print(f"Препроцессор сохранён: {preprocessor_path}")


Признаков после препроцессинга: 18
Препроцессор сохранён: /Users/bariatmamaeva/Desktop/uplift-marketing-mlops/models/preprocessor.joblib


# Split

In [6]:
processed_path = PROJECT_ROOT / config['data']['processed_path']

# Стратифицированное деление 
stratify_key = df['treatment'].astype(str) + df['conversion'].astype(str)
train_df, test_df = train_test_split(
    df,
    test_size=config['train']['test_size'],
    random_state=config['train']['random_state'],
    stratify=stratify_key
)

# Сохраняем данные
train_df.to_csv(processed_path / config['data']['train_file'], index=False)
test_df.to_csv(processed_path / config['data']['test_file'], index=False)

print(f"Данные сохранены в {processed_path}")
print(f" train: {len(train_df)} строк")
print(f" test: {len(test_df)} строк")

Данные сохранены в /Users/bariatmamaeva/Desktop/uplift-marketing-mlops/data/processed
 train: 51200 строк
 test: 12800 строк


# Train X-Learner with Optuna

In [7]:
feature_cols = [col for col in train_df.columns if col not in ['conversion', 'treatment', 'offer']]
X_train_feat = preprocessor.transform(train_df[feature_cols])
X_test_feat = preprocessor.transform(test_df[feature_cols])

t_train, t_test = train_df['treatment'], test_df['treatment']
y_train, y_test = train_df['conversion'], test_df['conversion']

# Оптимизация параметров через Optuna для effect модели
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', *config['model']['n_estimators_range']),
        'max_depth': trial.suggest_int('max_depth', *config['model']['max_depth_range']),
        'learning_rate': trial.suggest_float('learning_rate', *config['model']['learning_rate_range']),
        'num_leaves': trial.suggest_int('num_leaves', *config['model']['num_leaves_range']),
        'min_child_samples': trial.suggest_int('min_child_samples', *config['model']['min_child_samples_range']),
        'random_state': config['train']['random_state'],
        'n_jobs': -1
    }
    model = LGBMClassifier(**params)
    model.fit(X_train_feat[t_train==1], y_train[t_train==1])
    pred = model.predict_proba(X_test_feat[t_test==1])[:, 1]
    return roc_auc_score(y_test[t_test==1], pred)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=config['train']['optuna_trials'])

best_params = study.best_params
best_params.update({'random_state': config['train']['random_state'], 'n_jobs': -1})

# Сохраняем лучшие параметры
report_path = PROJECT_ROOT / config['folders']['report']
report_path.mkdir(parents=True, exist_ok=True)
with open(report_path / 'best_params.json', 'w') as f:
    json.dump(best_params, f, indent=2)

# Обучение X-Learner

# Модель вероятности назначения treatment
prop_model = LGBMClassifier(**best_params)
prop_model.fit(X_train_feat, t_train)

# Модель эффекта на treatment
effect_t = LGBMClassifier(**best_params)
effect_t.fit(X_train_feat[t_train==1], y_train[t_train==1])

# Модель эффекта на контроль
effect_c = LGBMClassifier(**best_params)
effect_c.fit(X_train_feat[t_train==0], y_train[t_train==0])

# Собираем в словарь и сохраняем
uplift_model_full = {
    'prop_model': prop_model,
    'effect_t': effect_t,
    'effect_c': effect_c
}

models_path = PROJECT_ROOT / config['folders']['models']
models_path.mkdir(parents=True, exist_ok=True)
joblib.dump(uplift_model_full, models_path / 'uplift_model.joblib')

# Сохраняем препроцессор
joblib.dump(preprocessor, models_path / 'preprocessor.joblib')

[I 2026-01-04 19:50:34,807] A new study created in memory with name: no-name-e552218e-ea2f-49bf-813c-cd17e71d7d71


[LightGBM] [Info] Number of positive: 5706, number of negative: 28449
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000693 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1064
[LightGBM] [Info] Number of data points in the train set: 34155, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.167062 -> initscore=-1.606595
[LightGBM] [Info] Start training from score -1.606595
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[I 2026-01-04 19:50:39,723] Trial 0 finished with value: 0.5963976828472609 and parameters: {'n_estimators': 337, 'max_depth': 11, 'learning_rate': 0.010883944244329964, 'num_leaves': 182, 'min_child_samples': 100}. Best is trial 0 with value: 0.5963976828472609.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Number of positive: 5706, number of negative: 28449
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000362 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1064
[LightGBM] [Info] Number of data points in the train set: 34155, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.167062 -> initscore=-1.606595
[LightGBM] [Info] Start training from score -1.606595
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

...

[I 2026-01-04 19:51:25,802] Trial 28 finished with value: 0.6142389071311067 and parameters: {'n_estimators': 142, 'max_depth': 8, 'learning_rate': 0.058866523588061964, 'num_leaves': 23, 'min_child_samples': 92}. Best is trial 28 with value: 0.6142389071311067.


[LightGBM] [Info] Number of positive: 5706, number of negative: 28449
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000450 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1064
[LightGBM] [Info] Number of data points in the train set: 34155, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.167062 -> initscore=-1.606595
[LightGBM] [Info] Start training from score -1.606595
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

[I 2026-01-04 19:51:27,183] Trial 29 finished with value: 0.6044442065167603 and parameters: {'n_estimators': 135, 'max_depth': 11, 'learning_rate': 0.026916290528722152, 'num_leaves': 85, 'min_child_samples': 94}. Best is trial 28 with value: 0.6142389071311067.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Number of positive: 34155, number of negative: 17045
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000564 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1064
[LightGBM] [Info] Number of data points in the train set: 51200, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.6

['/Users/bariatmamaeva/Desktop/uplift-marketing-mlops/models/preprocessor.joblib']

# Evaluate

In [8]:
feature_cols = [col for col in train_df.columns if col not in ['conversion', 'treatment', 'offer']]
X_train_feat = preprocessor.transform(train_df[feature_cols])
t_train, y_train = train_df['treatment'], train_df['conversion']

# Модель вероятности назначения treatment
prop_model = LGBMClassifier(**best_params)
prop_model.fit(X_train_feat, t_train)

# Модель эффекта для treatment
effect_t = LGBMClassifier(**best_params)
effect_t.fit(X_train_feat[t_train==1], y_train[t_train==1])

# Модель эффекта для control
effect_c = LGBMClassifier(**best_params)
effect_c.fit(X_train_feat[t_train==0], y_train[t_train==0])

# Сохраняем как словарь
uplift_model_full = {
    'prop_model': prop_model,
    'effect_t': effect_t,
    'effect_c': effect_c
}

# Сохраняем модель
models_path = PROJECT_ROOT / config['folders']['models']
models_path.mkdir(parents=True, exist_ok=True)
joblib.dump(uplift_model_full, models_path / 'uplift_model.joblib')


[LightGBM] [Info] Number of positive: 34155, number of negative: 17045
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000484 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1064
[LightGBM] [Info] Number of data points in the train set: 51200, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.667090 -> initscore=0.695052
[LightGBM] [Info] Start training from score 0.695052
[LightGBM] [Info] Number of positive: 5706, number of negative: 28449
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000376 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1064
[LightGBM] [Info] Number of data points in the train set: 34155, number of used features: 18
[LightGBM] [Info] [bi

['/Users/bariatmamaeva/Desktop/uplift-marketing-mlops/models/uplift_model.joblib']